# Finch usage

Finch is a WPS server focused on computing daily climate indicators, but also has a few utilities to facilitate data handling. To get started, first instantiate the client. 

In [1]:
from birdy import WPSClient
url = 'https://pavics.ouranos.ca/twitcher/ows/proxy/finch/wps'
wps = WPSClient(url)

The list of available processes can be displayed using the help function, as well as details about individual processes. 

In [2]:
help(wps)

Help on WPSClient in module birdy.client.base object:

class WPSClient(builtins.object)
 |  Returns a class where every public method is a WPS process available at
 |  the given url.
 |  
 |  Example:
 |      >>> emu = WPSClient(url='<server url>')
 |      >>> emu.hello('stranger')
 |      'Hello stranger'
 |  
 |  Methods defined here:
 |  
 |  __init__(self, url, processes=None, converters=None, username=None, password=None, headers=None, verify=True, cert=None, verbose=False, progress=False, version='1.0.0')
 |      Args:
 |          url (str): Link to WPS provider. config (Config): an instance
 |          processes: Specify a subset of processes to bind. Defaults to all
 |              processes.
 |          converters (dict): Correspondence of {mimetype: class} to convert
 |              this mimetype to a python object.
 |          username (str): passed to :class:`owslib.wps.WebProcessingService`
 |          password (str): passed to :class:`owslib.wps.WebProcessingService`
 |  

In [3]:
help(wps.frost_days)

Help on method frost_days in module birdy.client.base:

frost_days(tasmin=None, freq='YS') method of birdy.client.base.WPSClient instance
    Number of days where daily minimum temperatures are below 0.
    
    Parameters
    ----------
    tasmin : ComplexData:mimetype:`application/x-netcdf`
        NetCDF Files or archive (tar/zip) containing netCDF files.
    freq : {'YS', 'MS', 'QS-DEC', 'AS-JUL'}string
        Resampling frequency
    
    Returns
    -------
    output_netcdf : ComplexData:mimetype:`application/x-netcdf`
        The indicator values computed on the original input grid.
    output_log : ComplexData:mimetype:`text/plain`
        Collected logs during process run.



To actually compute an indicator, we need to specify the path to the netCDF file used as input for the calculation of the indicator. For example, to compute the number of `frost_days` per year, we need a time series of daily minimum temperature. Here we'll use a small test file. Note that here we're using a DAP link, but it could also be an URL to a NetCDF file, or the path to a local file on disk. We then simply call the indicator. The response is an object that can poll the server to inquire about the status of the process. This object can use two modes: 
 - synchronous: it will wait for the server's response before returning; or 
 - asynchronous: it will return immediately, but without the actual output from the process.
 
Here, since we're applying the process on a small test file, we're using the default synchronous mode. For long computations, use the asynchronous mode to avoid time-out errors. The asynchronous mode is activated by setting the `progress` attribute of the WPS client to True. 

In [4]:
tasmin = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/testdata/flyingpigeon/cmip3/tasmin.sresa2.miub_echo_g.run1.atm.da.nc"
resp = wps.frost_days(tasmin)

 owslib.wps.WPSException : {'code': 'ServerBusy', 'locator': '', 'text': 'Maximum number of parallel running processes reached. Please try later.'}


In [5]:
print(resp.status)
out = resp.get()
print(out)

ProcessSucceeded
frost_daysResponse(
    output_netcdf='http://localhost:5000/outputs/e30af6fa-7bfa-11e9-9d55-0242ac120017/out.nc',
    output_log='http://localhost:5000/outputs/e30af6fa-7bfa-11e9-9d55-0242ac120017/log.txt'
)


The `get` method returns a `NamedTuple` object with all the WPS outputs, either as references to files or actual content. To copy the file to the local disk, you can use the `getOutput` method. 

In [6]:
resp.getOutput('/tmp/out.nc')

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /outputs/e435307c-7bc8-11e9-9d55-0242ac120017/out.nc (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f604c3dacf8>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [ ]:
import xarray as xr
xr.open_dataset('/tmp/out.nc')

The birdy client offers a quicker way to download and open the files automatically using `asobj=True`, as long as the file format is known to birdy. 

In [ ]:
ds, log = resp.get(asobj=True)

In [ ]:
ds

In [ ]:
print(log)